<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Competitive_DS/blob/week_4/solution_4_4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

!pip install catboost -q
!pip install lightgbm -q
!pip install xgboost -q
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset
import xgboost as xgb
from xgboost import XGBRegressor

import pickle
import warnings
warnings.simplefilter('ignore')

import os
import platform
import psutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.0 MB/s eta 0:00:00


In [ ]:
print(platform.system(), platform.processor())
# print(os.uname())
print('CPU count:', os.cpu_count())
print('RAM (GB):', psutil.virtual_memory()[0]/1000000000)
print('RAM memory % used:', psutil.virtual_memory()[2])
!python -V

Linux x86_64
CPU count: 2
RAM (GB): 13.616316416
RAM memory % used: 9.6
Python 3.9.16


# Загрузка данных

In [ ]:
path_train = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv'
path_test = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv'

Оптимизируем объём памяти.

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
print('train')
train = import_data(path_train)

print('-' * 80)
print('test')
test = import_data(path_test)

train
Memory usage of dataframe is 0.30 MB
Memory usage after optimization is: 0.16 MB
Decreased by 47.7%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 0.23 MB
Memory usage after optimization is: 0.14 MB
Decreased by 40.9%


In [ ]:
train.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.779297,2015,76163,2021,110.00000,another_bug,4.738281,12141310.0,0.099976,180.875,0.023178,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.900391,2015,78218,2021,34.46875,electro_bug,4.480469,18039092.0,0.000000,187.875,12.304688,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.300781,2012,23340,2017,34.93750,gear_stick,4.769531,15883659.0,0.099976,102.375,2.513672,174,173


In [ ]:
train.deviation_normal_count.unique()

array([174], dtype=int16)

Этот признак никак не влияет на прогноз, поэтому не будем его использовать.

In [ ]:
print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}') 

train shape = (2337, 17),  test shape = (1913, 16)
train, test is null: (False, False)


In [ ]:
cat_features = ['model', 'car_type', 'fuel_type']
cat_features_indxs = [0, 1, 2]
targets = ['target_class', 'target_reg']
features2drop = ['car_id', 'deviation_normal_count'] 

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

print(f'\033[1mcat_features:\033[0m {cat_features}, \033[1mamount\033[0m = {len(cat_features)}')
print(f'\033[1mnum_features:\033[0m {num_features}, \033[1mamount\033[0m = {len(num_features)}')
print(f'\033[1mtargets:\033[0m {targets}, \033[1mamount\033[0m = {len(targets)}')

cat_features: ['model', 'car_type', 'fuel_type'], amount = 3
num_features: ['car_rating', 'year_to_start', 'riders', 'year_to_work', 'mean_rating', 'distance_sum', 'rating_min', 'speed_max', 'user_ride_quality_median', 'user_uniq'], amount = 10
targets: ['target_class', 'target_reg'], amount = 2


In [ ]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]

In [ ]:
# инициализируем случайное состояние, список лучших моделей, датафрейм для результатов
r_s = 42
n_est = 1000 
best_mdls = []
df_results = pd.DataFrame(columns=['model', 'mode', 'RMSE_best_fold', 'mean_RMSE', 'std'])

In [ ]:
# функция для реализации случайного поиска
def random_search_implementation(X, y, algorithm, init_params, grid, fit_params, cv=3, r_s=r_s):
    rgr = algorithm(**init_params)
    grid = grid

    model_regr = RandomizedSearchCV(estimator=rgr, 
                                    param_distributions=grid,
                                    cv=cv,
                                    scoring='neg_root_mean_squared_error',
                                    n_jobs=-1,
                                    verbose=100,
                                    n_iter=20,
                                    random_state=r_s)

    model_regr.fit(X, y, **fit_params)

    print('-' * 80)
    print(f'\n\033[1mbest parameters:\033[0m {model_regr.best_params_}')
    print(f'\033[1mbest score =\033[0m {round(abs(model_regr.best_score_), 4)}')
    # print(model_regr.get_params())
    return model_regr.best_params_ | init_params

In [ ]:
# функция для шага обучения
def lr_gen_simple(init_lr, epoch):    
    decay = init_lr / epoch
    lr_list = [init_lr/(1 + decay * i) for i in range(1, epoch + 1)]

    return lr_list  

In [ ]:
# функция для получения модели лучшего фолда
def best_fold_model(algorithm, X, y, params=None, cat_feats=None, n_splits=3,
                    r_s=42,n_rounds=1000, stop_rounds=200, init_lr=0.12):
    regrs = []
    RMSE_scores = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)
    
    for num, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if params != None:
            regr = algorithm(**params)
        else:
            regr = algorithm()
       
        # используем специализированные для каждого бустинга классы 
        if algorithm.__name__ == 'CatBoostRegressor':
            train_dataset = Pool(data=X_train, label=y_train,
                                 cat_features=cat_feats)        
            eval_dataset = Pool(data=X_test, label=y_test,
                                cat_features=cat_feats)
          
            regr.fit(train_dataset, 
                   eval_set=eval_dataset,
                   verbose=1000,
                   early_stopping_rounds=stop_rounds)      
          
        elif algorithm.__name__ == 'LGBMRegressor':
            train_dataset = Dataset(X_train, y_train, 
                                    categorical_feature=cat_feats,
                                    free_raw_data=False)
          
            eval_dataset = Dataset(X_test, y_test,
                                   categorical_feature=cat_feats,
                                   free_raw_data=False)
          
            regr = lgb.train(params, train_set=train_dataset, 
                             valid_sets=(eval_dataset),
                             categorical_feature=cat_feats,
                             callbacks=[lgb.early_stopping(stopping_rounds=stop_rounds), 
                                        lgb.reset_parameter(learning_rate=lr_gen_simple(init_lr, n_rounds))],
                             num_boost_round=n_rounds)
            
        elif algorithm.__name__ == 'XGBRegressor':
              train_dataset = xgb.DMatrix(X_train, label=y_train, nthread=-1,
                                          enable_categorical=True)
              
              eval_dataset = xgb.DMatrix(X_test, label=y_test, nthread=-1,
                                         enable_categorical=True)
              
              xgb.set_config(verbosity=0)

              regr = xgb.train(params,
                               dtrain=train_dataset,
                               evals=[(train_dataset, 'dtrain'),
                                      (eval_dataset, 'dtest')],
                               verbose_eval = False,
                               early_stopping_rounds=stop_rounds,
                               callbacks=[xgb.callback.LearningRateScheduler(lr_gen_simple(init_lr, n_rounds))],
                               num_boost_round=n_rounds)
              
              X_test = eval_dataset
          
        regrs.append(regr)

        y_pred = regr.predict(X_test)
        # выводим номер фолда и значение RMSE
        RMSE = mean_squared_error(y_test.values, y_pred, squared=False)
        print(f'fold: {num}, test_RMSE: {RMSE}')
        # выводим номер лучшего фолда
        RMSE_scores.append(RMSE)
        min_RMSE = min(RMSE_scores)
        bst = regrs[np.argmin(RMSE_scores)]
        print(f'Best fold number = {np.argmin(RMSE_scores)}\n')
        
    assert len(regrs) == n_splits

    print('-' * 80)    
    print(f'Среднее значение метрики по трём фолдам \033[1mtest_RMSE_mean\033[0m = {np.mean(RMSE_scores).round(4)}')

    return bst, algorithm.__name__, min_RMSE, np.mean(RMSE_scores).round(4), np.std(RMSE_scores).round(4)

# Основная часть

## CatBoostRegressor (random state = 42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=r_s)

In [ ]:
%%time

grid = {'depth': np.arange(2, 12, 1),        
        'learning_rate' : [.02, .04, .06, .08, 0.1, .12],
        'min_data_in_leaf': np.arange(100, 500, 50),
        'l2_leaf_reg': np.arange(2, 12, 2),
        'max_bin': np.arange(100, 300, 10),
        'subsample': [.7, .75, .8, .85, .9, .95]}

init_params = {'loss_function': 'RMSE',
               'eval_metric': 'RMSE',
               'random_seed': r_s,
               'thread_count': -1,
               'verbose': 0,
               'task_type': "CPU",
               'bootstrap_type': "Bernoulli",
               'random_strength': 1,
               'n_estimators': n_est}

fit_params = {'cat_features': cat_features,
              'eval_set': [(X_test, y_test)],
              'verbose': 0,      
              'early_stopping_rounds': 200}

best_catboost_params = random_search_implementation(X_train, y_train,
                                                    CatBoostRegressor,
                                                    init_params, grid,
                                                    fit_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
--------------------------------------------------------------------------------

best parameters: {'subsample': 0.95, 'min_data_in_leaf': 350, 'max_bin': 290, 'learning_rate': 0.06, 'l2_leaf_reg': 2, 'depth': 7}
best score = 11.664
CPU times: user 3.31 s, sys: 784 ms, total: 4.09 s
Wall time: 3min 41s


In [ ]:
res = best_fold_model(CatBoostRegressor, X, y, best_catboost_params,
                      cat_features)
best_mdls.append(res[0])

0:	learn: 17.3245548	test: 16.7341895	best: 16.7341895 (0)	total: 4.25ms	remaining: 4.24s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.91224505
bestIteration = 83

Shrink model to first 84 iterations.
fold: 0, test_RMSE: 11.912245046646174
Best fold number = 0

0:	learn: 16.7524580	test: 18.0058200	best: 18.0058200 (0)	total: 4.45ms	remaining: 4.44s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.96002421
bestIteration = 75

Shrink model to first 76 iterations.
fold: 1, test_RMSE: 11.96002421114515
Best fold number = 0

0:	learn: 17.3601904	test: 16.7730680	best: 16.7730680 (0)	total: 3.96ms	remaining: 3.96s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.37458889
bestIteration = 159

Shrink model to first 160 iterations.
fold: 2, test_RMSE: 11.374588887703041
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean = 

In [ ]:
df_results = df_results.append({'model': res[1],
                                'mode': 'random_state = 42',
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4]}, ignore_index=True)

## LightGBM ('goss')

In [ ]:
X_le = X.copy()

In [ ]:
le = LabelEncoder()
for col in cat_features:
    X_le[col] = le.fit_transform(X_le[col])

In [ ]:
X_le_train, X_le_test, y_le_train, y_le_test = train_test_split(X_le, y, test_size=0.2, random_state=r_s)

In [ ]:
%%time

grid = {'max_depth': np.arange(2, 12, 1),
        'min_data_in_leaf': np.arange(1, 30),
        'lambda_l2': np.arange(2, 12, 2),
        'max_bin': np.arange(10, 100, 10),
        'subsample': [.75, .85, .95],         
        'num_leaves': np.arange(1, 10),
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 0.95]}
      
init_params = {'boosting_type': 'goss',
               'random_state': r_s,
               'n_jobs':-1,
               'n_estimators': n_est,
               'metric': 'rmse',
               'objective': 'rmse',
               'random_state': r_s,
               'verbose': -1,
               'device': 'cpu'}

fit_params = {'eval_metric': 'rmse',
              'eval_set': [(X_le_test, y_le_test)],
              'verbose': False,
              'categorical_feature': cat_features_indxs,
              'callbacks': [lgb.early_stopping(stopping_rounds=50), 
                            lgb.reset_parameter(learning_rate=lr_gen_simple(.02, n_est))]}

best_LGBM_params = random_search_implementation(X_le_train, y_le_train, 
                                                LGBMRegressor, init_params,
                                                grid, fit_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[269]	valid_0's rmse: 11.9474
--------------------------------------------------------------------------------

best parameters: {'subsample': 0.95, 'num_leaves': 9, 'min_data_in_leaf': 8, 'max_depth': 11, 'max_bin': 60, 'lambda_l2': 2, 'colsample_bytree': 0.9}
best score = 11.7661
CPU times: user 3.83 s, sys: 38.6 ms, total: 3.87 s
Wall time: 20.5 s


In [ ]:
res = best_fold_model(LGBMRegressor, X_le, y, best_LGBM_params, cat_features,
                      n_rounds=n_est, stop_rounds=50, init_lr=.02)
best_mdls.append(res[0])

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[264]	valid_0's rmse: 11.8344
fold: 0, test_RMSE: 11.834366492776924
Best fold number = 0

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[285]	valid_0's rmse: 11.9343
fold: 1, test_RMSE: 11.934273166899315
Best fold number = 0

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[237]	valid_0's rmse: 11.5111
fold: 2, test_RMSE: 11.511082614573594
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean = 11.7599


In [ ]:
df_results = df_results.append({'model': res[1],
                                'mode': 'goss',
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4]}, ignore_index=True)

## XGBoost ('dart')

In [ ]:
%%time

grid = {'tree_method': ['hist', 'approx'],
        'max_depth': np.arange(2, 12, 1),
        'max_leaves': np.arange(1, 10),
        'lambda': np.arange(1, 10, 1),
        'max_bin': np.arange(10, 100, 10),
        'subsample': [.75, .85, .95],
        'alpha': np.arange(0, 10, 1),
        'gamma': np.linspace(0, 1, 3),
        'min_child_weight': np.linspace(1, 10, 10),
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 0.95],
        'rate_drop':np.linspace(0.5, 1, 4),
        'skip_drop': np.linspace(0.5, 1, 4)}
      
init_params = {'eval_metric':'rmse',
               'enable_categorical': True,
               'booster': 'dart',
               'random_state': r_s,
               'n_jobs':-1,
               'n_estimators': n_est,
               'objective': 'reg:squarederror',
               'early_stopping_rounds': 20,
               'callbacks': [xgb.callback.LearningRateScheduler(lr_gen_simple(.04, n_est))]}

fit_params = {'eval_set': [(X_test, y_test)],              
              'verbose': 0}

best_XGB_params = random_search_implementation(X_train, y_train, 
                                               XGBRegressor, init_params,
                                               grid, fit_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
--------------------------------------------------------------------------------

best parameters: {'tree_method': 'hist', 'subsample': 0.95, 'skip_drop': 0.6666666666666666, 'rate_drop': 0.5, 'min_child_weight': 4.0, 'max_leaves': 8, 'max_depth': 6, 'max_bin': 40, 'lambda': 4, 'gamma': 0.5, 'colsample_bytree': 0.8, 'alpha': 9}
best score = 11.7449
CPU times: user 10.4 s, sys: 1.48 s, total: 11.9 s
Wall time: 3min 50s


In [ ]:
res = best_fold_model(XGBRegressor, X, y, best_XGB_params,
                      n_rounds=n_est, stop_rounds=20, init_lr=.04)
best_mdls.append(res[0])

fold: 0, test_RMSE: 11.761449813842773
Best fold number = 0

fold: 1, test_RMSE: 11.842194557189941
Best fold number = 0

fold: 2, test_RMSE: 11.357574462890625
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean = 11.65369987487793


In [ ]:
df_results = df_results.append({'model': res[1],
                                'mode': 'dart',
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4]}, ignore_index=True)

# Дополнительные модели

## CatBoostRegressor (random state = 7575)

In [ ]:
%%time

grid = {'depth': np.arange(2, 12, 1),        
        'learning_rate' : [.02, .04, .06, .08, 0.1, .12],
        'min_data_in_leaf': np.arange(100, 500, 50),
        'l2_leaf_reg': np.arange(2, 12, 2),
        'max_bin': np.arange(100, 300, 10),
        'subsample': [.7, .75, .8, .85, .9, .95]}

init_params = {'loss_function': 'RMSE',
               'eval_metric': 'RMSE',
               'random_seed': 7575,
               'thread_count': -1,
               'verbose': 0,
               'task_type': "CPU",
               'bootstrap_type': "Bernoulli",
               'random_strength': 1,
               'n_estimators': n_est}

fit_params = {'cat_features': cat_features,
              'eval_set': [(X_test, y_test)],
              'verbose': 0,      
              'early_stopping_rounds': 200}

best_catboost_params = random_search_implementation(X_train, y_train, CatBoostRegressor, init_params, grid, fit_params)   

Fitting 3 folds for each of 20 candidates, totalling 60 fits
--------------------------------------------------------------------------------

best parameters: {'subsample': 0.95, 'min_data_in_leaf': 250, 'max_bin': 270, 'learning_rate': 0.1, 'l2_leaf_reg': 2, 'depth': 6}
best score = 11.6219
CPU times: user 2.52 s, sys: 200 ms, total: 2.72 s
Wall time: 3min 19s


In [ ]:
res = best_fold_model(CatBoostRegressor, X, y, best_catboost_params, cat_features)
best_mdls.append(res[0])

0:	learn: 16.9259015	test: 16.3551890	best: 16.3551890 (0)	total: 3.03ms	remaining: 3.03s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.86331758
bestIteration = 44

Shrink model to first 45 iterations.
fold: 0, test_RMSE: 11.863317583672519
Best fold number = 0

0:	learn: 16.4215937	test: 17.6510416	best: 17.6510416 (0)	total: 8.42ms	remaining: 8.41s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.83268471
bestIteration = 178

Shrink model to first 179 iterations.
fold: 1, test_RMSE: 11.832684712859827
Best fold number = 1

0:	learn: 17.1079297	test: 16.5278394	best: 16.5278394 (0)	total: 3.21ms	remaining: 3.21s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.40168631
bestIteration = 142

Shrink model to first 143 iterations.
fold: 2, test_RMSE: 11.401686313691192
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean

In [ ]:
df_results = df_results.append({'model': res[1],
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4],
                                'mode': 'random_state = 7575'}, ignore_index=True)

## LightGBM ('dart')

In [ ]:
%%time

grid = {'max_depth': np.arange(2, 12, 1),
        'min_data_in_leaf': np.arange(1, 30),
        'lambda_l2': np.arange(2, 12, 2),
        'max_bin': np.arange(10, 100, 10),
        'subsample': [.75, .85, .95],         
        'num_leaves': np.arange(1, 10),
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 0.95]}
      
init_params = {'boosting_type': 'dart',
               'random_state': r_s,
               'n_jobs':-1,
               'n_estimators': n_est,
               'metric': 'rmse',
               'objective': 'rmse',
               'random_state': r_s,
               'verbose': -1,
               'device': 'cpu'}

fit_params = {'eval_metric': 'rmse',
              'eval_set': [(X_le_test, y_le_test)],
              'verbose': False,
              'categorical_feature': cat_features_indxs,
              'callbacks': [lgb.early_stopping(stopping_rounds=200), 
                            lgb.reset_parameter(learning_rate=lr_gen_simple(.7, n_est))]}

best_LGBM_params = random_search_implementation(X_le_train, y_le_train, 
                                                LGBMRegressor, init_params,
                                                grid, fit_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
--------------------------------------------------------------------------------

best parameters: {'subsample': 0.75, 'num_leaves': 8, 'min_data_in_leaf': 3, 'max_depth': 7, 'max_bin': 40, 'lambda_l2': 2, 'colsample_bytree': 0.8}
best score = 11.7207
CPU times: user 12.2 s, sys: 276 ms, total: 12.4 s
Wall time: 2min 5s


In [ ]:
res = best_fold_model(LGBMRegressor, X_le, y, best_LGBM_params, cat_features,
                      n_rounds=1000, init_lr=.7)
best_mdls.append(res[0])

fold: 0, test_RMSE: 11.95384357755469
Best fold number = 0

fold: 1, test_RMSE: 11.920302982871021
Best fold number = 1

fold: 2, test_RMSE: 11.511305955545428
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean = 11.7952


In [ ]:
df_results = df_results.append({'model': res[1],
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4],
                                'mode': 'dart'}, ignore_index=True)

## XGBoost ('gbtree')

In [ ]:
%%time

grid = {'tree_method': ['hist', 'approx', 'exact'],
        'max_depth': np.arange(2, 12, 1),
        'max_leaves': np.arange(1, 10),
        'lambda': np.arange(1, 10, 1),
        'max_bin': np.arange(10, 100, 10),
        'subsample': [.75, .85, .95],
        'alpha': np.arange(0, 10, 1),
        'gamma': np.linspace(0, 1, 10),
        'min_child_weight': np.linspace(1, 10, 10),
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 0.95]}
      
init_params = {'eval_metric':'rmse',
               'enable_categorical': True,
               'booster': 'gbtree',
               'random_state': r_s,
               'n_jobs':-1,
               'n_estimators': n_est,
               'objective': 'reg:squarederror',
               'callbacks': [xgb.callback.LearningRateScheduler(lr_gen_simple(.01, n_est))],
               'early_stopping_rounds': 200}

fit_params = {'eval_set': [(X_test, y_test)],              
              'verbose': False}

best_XGB_params = random_search_implementation(X_train, y_train, 
                                               XGBRegressor, init_params,
                                               grid, fit_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
--------------------------------------------------------------------------------

best parameters: {'tree_method': 'approx', 'subsample': 0.95, 'min_child_weight': 8.0, 'max_leaves': 7, 'max_depth': 7, 'max_bin': 90, 'lambda': 3, 'gamma': 0.2222222222222222, 'colsample_bytree': 0.9, 'alpha': 7}
best score = 11.7349
CPU times: user 1.85 s, sys: 76.6 ms, total: 1.93 s
Wall time: 12.8 s


In [ ]:
res = best_fold_model(XGBRegressor, X, y, best_XGB_params,
                      n_rounds=n_est, init_lr=.01)
best_mdls.append(res[0])

fold: 0, test_RMSE: 11.837638854980469
Best fold number = 0

fold: 1, test_RMSE: 11.901158332824707
Best fold number = 0

fold: 2, test_RMSE: 11.483156204223633
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по трём фолдам test_RMSE_mean = 11.740699768066406


In [ ]:
df_results = df_results.append({'model': res[1],
                                'RMSE_best_fold': res[2],
                                'mean_RMSE': res[3],
                                'std': res[4],
                                'mode': 'gbtree'}, ignore_index=True)

# Сохранение моделей и прогнозирование

Сохраним полученные модели

In [ ]:
def prediction(i, X):
    with open(f"model_{i}.pkl", "rb") as f:
        model = pickle.load(f)

    return model.predict(X)   

In [ ]:
for i, mdl in enumerate(best_mdls):
    with open(f'model_{i}.pkl', "wb") as f:
        pickle.dump(mdl, f)

Получим прогнозы для трёх основных моделей

In [ ]:
# подготовка submission для разных моделей
X_sub = test[filtered_features].drop(targets, axis=1, errors="ignore")

X_le_sub = X_sub.copy()

for col in cat_features:
    X_le_sub[col] = le.fit_transform(X_sub[col])

X_sub_dmatrix = xgb.DMatrix(X_sub, nthread=-1, enable_categorical=True)

## CatBoostRegressor (random state = 7575)

In [ ]:
df_sub = pd.DataFrame(test['car_id'])
df_sub['CtB_target_reg']=(prediction(0, X_sub))

## LightGBM ('goss')

In [ ]:
df_sub['LGBM_target_reg'] = prediction(1, X_le_sub)

## XGBoost ('dart')

In [ ]:
df_sub['XGB_target_reg'] = prediction(2, X_sub_dmatrix)

In [ ]:
df_sub.head(3)

,car_id,CtB_target_reg,LGBM_target_reg,XGB_target_reg
0,P17494612l,41.348844,45.197706,47.434601
1,N-1530212S,32.281348,35.633145,35.873508
2,B-1154399t,33.456254,34.685609,31.681417


## Усреднённый прогноз

In [ ]:
temp_columns = ['CtB_target_reg',	'LGBM_target_reg', 'XGB_target_reg']

In [ ]:
df_sub['target_reg'] = df_sub.mean(axis=1)
df_sub.drop(temp_columns, axis=1, inplace=True)
df_sub = df_sub.reset_index(drop=True)
df_sub.head(3)

,car_id,target_reg
0,P17494612l,44.660384
1,N-1530212S,34.596000
2,B-1154399t,33.274427


In [ ]:
df_sub.to_csv("try_1.csv", index=False)

# Результаты и выводы.

In [ ]:
df_results

,model,mode,RMSE_best_fold,mean_RMSE,std
0,CatBoostRegressor,random_state = 42,11.374589,11.7490,0.2654
1,LGBMRegressor,goss,11.511083,11.7599,0.1806
2,XGBRegressor,dart,11.357574,11.6537,0.2120
3,CatBoostRegressor,random_state = 7575,11.401686,11.6992,0.2108
4,LGBMRegressor,dart,11.511306,11.7952,0.2012
5,XGBRegressor,gbtree,11.483156,11.7407,0.1839


* Были проведены обучены три бустинга с различными параметрами.
* Был проведён подбор гиперпараметров. Использовался random search, а не grid search из-за ограничения работы ноутбука по времени. Можно было бы выбирать лучшие фолды из random search, но тогда не использовались бы специализированные классы(Pool, Dmatrix, Dataset) для разных бустингов. В catboost возможно было воспользоваться "батарейкой". Но решил не использовать для единообразия решения.
* Модели с лучшими параметрами были обучены на трёх фолдах и были выбраны лучшие фолды от каждой модели. Здесь и реализована работа с внутренними классами для каждого бустинга.
* В моделях на основе LightGBM и XGBoost использался самый простой самописный изменяемый шаг обучения. В catboost, насколько я понял, такой возможности не реализовано.
* Везде использовались early_stopping и многопоточность.
* Использовалась оптимизация по использованию памяти для датасета. Там же категориальные признаки были переведены в тип category. Это использовалось для моделей на основе XGboost. Catboost работает с категориальными признаками напрямую поэтому туда можно было подавать только список признаков. Для LightGBM использовался encoding категориальных признаков.
* Итоговое предсказание - усреднение трёх лучших моделей.
* В принципе, все модели показали достаточно близкие результаты. По моим измерениям медленнее всего отрабатывали модели на основе catboost. Быстрее всего отрабатывали модели на основе LightGBM. Ноутбук готовился в colab, поэтому время выполнения ячеек может отличаться от pdf.